In [15]:
import os
import re
import torch
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from collections import Counter

# Inicializar CodeBERT
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

# Anonimización básica
def anonymizar_codigo(code):
    tokens = re.findall(r'\b[a-zA-Z_][a-zA-Z0-9_]*\b', code)
    usados = {}
    nuevo_codigo = code
    contador = 1
    for tok in tokens:
        if tok not in usados and tok not in {"def", "if", "else", "for", "while", "return", "print", "input"}:
            usados[tok] = f"VAR_{contador}"
            contador += 1
    for original, nuevo in usados.items():
        nuevo_codigo = re.sub(rf'\b{original}\b', nuevo, nuevo_codigo)
    return nuevo_codigo

# Extraer función principal del archivo
def extraer_funcion_principal(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        code = f.read()
    funciones = re.findall(r"(def\s+[a-zA-Z_][a-zA-Z0-9_]*\(.*?\):(?:\n(?:\s{4}|\t).*)*)", code)
    return funciones[0] if funciones else code  # Si no hay funciones, usar todo

# Embedding con CodeBERT
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# Pipeline principal
def entrenar_clasificador(csv_path):
    df = pd.read_csv(csv_path)
    X, y = [], []

    for _, row in df.iterrows():
        path1, path2, etiqueta = row["codigo_1"], row["codigo_2"], row["tipo_plagio"]

        if not os.path.exists(path1) or not os.path.exists(path2):
            print(f"❌ Archivo no encontrado: {path1} o {path2}")
            continue

        cod1 = anonymizar_codigo(extraer_funcion_principal(path1))
        cod2 = anonymizar_codigo(extraer_funcion_principal(path2))

        emb1 = get_embedding(cod1)
        emb2 = get_embedding(cod2)

        vector = np.concatenate([emb1, emb2, np.abs(emb1 - emb2)])
        X.append(vector)
        y.append(etiqueta)

    print(f"\nPares cargados: {len(X)}")
    print("Distribución de clases:", Counter(y))

    if len(X) >= 4:
        try:
            if all(v >= 2 for v in Counter(y).values()):
                print("✅ Usando stratify para dividir train/test")
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
            else:
                print("⚠️ Pocas muestras en alguna clase. Usando split sin stratify.")
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        except:
            print("⚠️ Error al dividir con stratify. Reintentando sin stratify.")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        clf = RandomForestClassifier()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        print("\n=== Reporte de Clasificación ===")
        print(classification_report(y_test, y_pred))
    else:
        print("⚠️ No hay suficientes ejemplos para entrenar el modelo.")

# Ejecutar
entrenar_clasificador("pares.csv")


❌ Archivo no encontrado: dataset/par-29-a-Treesort.py o dataset/par-29-e-Treesort.py

Pares cargados: 122
Distribución de clases: Counter({1: 31, 3: 31, 4: 30, 2: 30})
✅ Usando stratify para dividir train/test

=== Reporte de Clasificación ===
              precision    recall  f1-score   support

           1       0.47      0.78      0.58         9
           2       0.00      0.00      0.00         9
           3       0.12      0.10      0.11        10
           4       0.33      0.44      0.38         9

    accuracy                           0.32        37
   macro avg       0.23      0.33      0.27        37
weighted avg       0.23      0.32      0.26        37



In [5]:
import os
import re
import torch
import numpy as np
from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity

# === Inicializar CodeBERT ===
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

# === Función para anonimizar código ===
def anonimizar_codigo(code):
    tokens = re.findall(r'\b[a-zA-Z_][a-zA-Z0-9_]*\b', code)
    usados = {}
    nuevo_codigo = code
    contador = 1
    for tok in tokens:
        if tok not in usados and tok not in {"def", "if", "else", "for", "while", "return", "print", "input"}:
            usados[tok] = f"VAR_{contador}"
            contador += 1
    for original, nuevo in usados.items():
        nuevo_codigo = re.sub(rf'\b{original}\b', nuevo, nuevo_codigo)
    return nuevo_codigo

# === Extraer todas las funciones de un archivo ===
def extraer_todas_funciones(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        code = f.read()
    funciones = re.findall(r"(def\s+[a-zA-Z_][a-zA-Z0-9_]*\(.*?\):(?:\n(?:\s{4}|\t).*)*)", code)
    return funciones if funciones else [code]

# === Obtener embedding de una función ===
def obtener_embedding(texto):
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# === Comparar funciones entre dos archivos ===
def comparar_funciones(path1, path2):
    funciones_1 = extraer_todas_funciones(path1)
    funciones_2 = extraer_todas_funciones(path2)

    print(f"🔍 {len(funciones_1)} funciones en {path1}")
    print(f"🔍 {len(funciones_2)} funciones en {path2}")

    emb_1 = [obtener_embedding(anonimizar_codigo(f)) for f in funciones_1]
    emb_2 = [obtener_embedding(anonimizar_codigo(f)) for f in funciones_2]

    similitudes = []
    for i, e1 in enumerate(emb_1):
        fila = []
        for j, e2 in enumerate(emb_2):
            sim = cosine_similarity([e1], [e2])[0][0]
            fila.append(sim)
            print(f"Similitud f{i} (A) vs f{j} (B): {sim:.4f}")
        similitudes.append(fila)

    return np.array(similitudes)

# === Ejemplo de uso ===
if __name__ == "__main__":
    archivo_a = "dataset/par-10-a-bubble-sort.py"
    archivo_b = "dataset/prueba.cpp"
    matriz = comparar_funciones(archivo_a, archivo_b)

    print("\n🧾 Matriz de similitud:")
    print(np.round(matriz, 4))


🔍 2 funciones en dataset/par-10-a-bubble-sort.py
🔍 1 funciones en dataset/prueba.cpp
Similitud f0 (A) vs f0 (B): 0.9805
Similitud f1 (A) vs f0 (B): 0.9850

🧾 Matriz de similitud:
[[0.9805]
 [0.985 ]]


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
from scipy.spatial.distance import euclidean

# Asegúrate de tener definidas estas funciones (ya las tienes):
# - anonimizar_codigo()
# - extraer_todas_funciones()
# - obtener_embedding()

def features_por_par(path1, path2):
    if not os.path.exists(path1) or not os.path.exists(path2):
        return None

    funcs1 = extraer_todas_funciones(path1)
    funcs2 = extraer_todas_funciones(path2)

    emb1 = [obtener_embedding(anonimizar_codigo(f)) for f in funcs1]
    emb2 = [obtener_embedding(anonimizar_codigo(f)) for f in funcs2]

    if not emb1 or not emb2:
        return None

    # Cambia cosine_similarity por euclidean
    distancias = [-euclidean(e1, e2) for e1 in emb1 for e2 in emb2]

    max_sim = max(distancias)
    avg_sim = np.mean(distancias)
    count_95 = sum(1 for s in distancias if s > -0.05)
    count_90 = sum(1 for s in distancias if s > -0.1)

    return [max_sim, avg_sim, count_95, count_90, len(emb1), len(emb2)]


def entrenar_modelo_funcion_por_funcion(csv_path):
    df = pd.read_csv(csv_path)
    X, y = [], []

    for _, row in df.iterrows():
        path1, path2, label = row["codigo_1"], row["codigo_2"], row["tipo_plagio"]
        feats = features_por_par(path1, path2)
        if feats is not None:
            X.append(feats)
            y.append(label)

    print(f"Pares válidos: {len(X)}")
    print("Distribución:", Counter(y))

    if len(set(y)) > 1:
        try:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
        except ValueError:
            print("⚠️ Stratify no posible, dividiendo sin estratificación.")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        clf = KNeighborsClassifier(n_neighbors=15)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        print("\n=== Reporte de Clasificación (KNN) ===")
        print(classification_report(y_test, y_pred))
    else:
        print("⚠️ Solo hay una clase en el dataset. No se puede entrenar.")

entrenar_modelo_funcion_por_funcion("pares.csv")



Pares válidos: 237
Distribución: Counter({2: 71, 3: 70, 1: 66, 4: 30})

=== Reporte de Clasificación (KNN) ===
              precision    recall  f1-score   support

           1       0.54      0.70      0.61        20
           2       0.32      0.32      0.32        22
           3       0.61      0.52      0.56        21
           4       0.33      0.22      0.27         9

    accuracy                           0.47        72
   macro avg       0.45      0.44      0.44        72
weighted avg       0.47      0.47      0.46        72

